<a href="https://colab.research.google.com/github/drshahizan/Python-big-data/blob/main/Project/Rojak/Rojak_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**15 Million Chess Games from Lichess 2013-2014 Analysis and Libraries Comparisons**

###Group Members: 

<table>
  <tr>
    <th>Name</th>
    <th>Matric No</th>
  </tr>
  <tr>
    <th>Eddie Wong Chung Pheng </th>
    <th>A20EC0031</th>
  </tr>
  <tr>
    <th>Madihah Binti Che Zabri </th>
    <th>A20EC0074</th>
  </tr>
  <tr>
    <th>Nurarissa Dayana Binti Mohd Sukri</th>
    <th>A20EC0120</th>
  </tr>
  <tr> 
    <th>Vincent Boo Ee Khai</th>
    <th>A20EC0231</th>
  </tr>
</table>
<br></br>

## About the dataset

*  This dataset contains data from all rated games played in Lichess from January 2013 to December 2014, in total there are around 15.000.000 games.
*  The dataset consists of 1048576 rows and 16 columns.
*  The size of this dataset is 1.99GB.
*  The dataset can be obtained from [here](https://www.kaggle.com/datasets/maca11/chess-games-from-lichess-20132014?select=Lichess_2013_2014_Complete.csv)





In [1]:
# Read dataset from google drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/file/d/10CIeiPoyqyyDT_4C2B0eGANY-XVFpzkr/view?usp=share_link'
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Lichess_2013_2014_Complete.csv') 

Install Required Library

In [2]:
!pip install vaex
!pip install koalas
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import vaex as vx
import databricks.koalas as kl
import random
from sklearn.linear_model import LogisticRegression

In [5]:
%time df = pd.read_csv("Lichess_2013_2014_Complete.csv")
pd.set_option('display.max_columns', None)

CPU times: user 44.5 s, sys: 5.95 s, total: 50.4 s
Wall time: 59.9 s


In [6]:
%time df.head(5)

CPU times: user 331 µs, sys: 0 ns, total: 331 µs
Wall time: 337 µs


,WhiteElo,BlackElo,WhiteName,BlackName,Winner,Termination,Site,Day,Month,Year,InitialTime,Increment,TimeControl,Opening,ECO,Number_of_Moves
0,1639,1403,BFG9k,mamalak,White,Normal,https://lichess.org/j1dkb5dw,31,12,2012,600,8,Rapid,French Defense: Normal Variation,C00,13
1,1654,1919,Desmond_Wilson,savinka59,White,Normal,https://lichess.org/a9tcp02g,31,12,2012,480,2,Rapid,"Queen's Pawn Game: Colle System, Anti-Colle",D04,18
2,1643,1747,Kozakmamay007,VanillaShamanilla,White,Normal,https://lichess.org/szom2tog,31,12,2012,420,17,Rapid,Four Knights Game: Italian Variation,C50,11
3,1824,1973,Naitero_Nagasaki,800,Black,Normal,https://lichess.org/rklpc7mk,31,12,2012,60,1,Bullet,Caro-Kann Defense: Goldman Variation,B12,47
4,1765,1815,nichiren1967,Naitero_Nagasaki,Black,Normal,https://lichess.org/1xb3os63,31,12,2012,60,1,Bullet,French Defense: La Bourdonnais Variation,C00,23


In [7]:
%time vaex_df = vx.open('Lichess_2013_2014_Complete.csv') 

CPU times: user 2.64 s, sys: 1.48 s, total: 4.12 s
Wall time: 10.3 s


In [8]:
%time vaex_df.head(5)

CPU times: user 1.47 ms, sys: 0 ns, total: 1.47 ms
Wall time: 4.99 ms


#,WhiteElo,BlackElo,WhiteName,BlackName,Winner,Termination,Site,Day,Month,Year,InitialTime,Increment,TimeControl,Opening,ECO,Number_of_Moves
0,1639,1403,BFG9k,mamalak,White,Normal,https://lichess.org/j1dkb5dw,31,12,2012,600,8,Rapid,French Defense: Normal Variation,C00,13
1,1654,1919,Desmond_Wilson,savinka59,White,Normal,https://lichess.org/a9tcp02g,31,12,2012,480,2,Rapid,"Queen's Pawn Game: Colle System, Anti-Colle",D04,18
2,1643,1747,Kozakmamay007,VanillaShamanilla,White,Normal,https://lichess.org/szom2tog,31,12,2012,420,17,Rapid,Four Knights Game: Italian Variation,C50,11
3,1824,1973,Naitero_Nagasaki,800,Black,Normal,https://lichess.org/rklpc7mk,31,12,2012,60,1,Bullet,Caro-Kann Defense: Goldman Variation,B12,47
4,1765,1815,nichiren1967,Naitero_Nagasaki,Black,Normal,https://lichess.org/1xb3os63,31,12,2012,60,1,Bullet,French Defense: La Bourdonnais Variation,C00,23


In [9]:
%time koalas_df = kl.read_csv('Lichess_2013_2014_Complete.csv')

CPU times: user 665 ms, sys: 259 ms, total: 924 ms
Wall time: 1min 19s


In [10]:
%time koalas_df.head(5)

CPU times: user 54.9 ms, sys: 11.5 ms, total: 66.5 ms
Wall time: 590 ms


,WhiteElo,BlackElo,WhiteName,BlackName,Winner,Termination,Site,Day,Month,Year,InitialTime,Increment,TimeControl,Opening,ECO,Number_of_Moves
0,1639,1403,BFG9k,mamalak,White,Normal,https://lichess.org/j1dkb5dw,31,12,2012,600,8,Rapid,French Defense: Normal Variation,C00,13
1,1654,1919,Desmond_Wilson,savinka59,White,Normal,https://lichess.org/a9tcp02g,31,12,2012,480,2,Rapid,"Queen's Pawn Game: Colle System, Anti-Colle",D04,18
2,1643,1747,Kozakmamay007,VanillaShamanilla,White,Normal,https://lichess.org/szom2tog,31,12,2012,420,17,Rapid,Four Knights Game: Italian Variation,C50,11
3,1824,1973,Naitero_Nagasaki,800,Black,Normal,https://lichess.org/rklpc7mk,31,12,2012,60,1,Bullet,Caro-Kann Defense: Goldman Variation,B12,47
4,1765,1815,nichiren1967,Naitero_Nagasaki,Black,Normal,https://lichess.org/1xb3os63,31,12,2012,60,1,Bullet,French Defense: La Bourdonnais Variation,C00,23


In [11]:
df.dtypes

WhiteElo            int64
BlackElo            int64
WhiteName          object
BlackName          object
Winner             object
Termination        object
Site               object
Day                 int64
Month               int64
Year                int64
InitialTime         int64
Increment           int64
TimeControl        object
Opening            object
ECO                object
Number_of_Moves     int64
dtype: object

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14918230 entries, 0 to 14918229
Data columns (total 16 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   WhiteElo         int64 
 1   BlackElo         int64 
 2   WhiteName        object
 3   BlackName        object
 4   Winner           object
 5   Termination      object
 6   Site             object
 7   Day              int64 
 8   Month            int64 
 9   Year             int64 
 10  InitialTime      int64 
 11  Increment        int64 
 12  TimeControl      object
 13  Opening          object
 14  ECO              object
 15  Number_of_Moves  int64 
dtypes: int64(8), object(8)
memory usage: 1.8+ GB


In [13]:
df.shape

(14918230, 16)

In [14]:
# Count the number of rows
df.count()     

WhiteElo           14918230
BlackElo           14918230
WhiteName          14918230
BlackName          14918230
Winner             14918230
Termination        14918230
Site               14918230
Day                14918230
Month              14918230
Year               14918230
InitialTime        14918230
Increment          14918230
TimeControl        14918230
Opening            14918230
ECO                14918230
Number_of_Moves    14918230
dtype: int64

In [11]:
df.isnull().sum()

WhiteElo           0
BlackElo           0
WhiteName          0
BlackName          0
Winner             0
Termination        0
Site               0
Day                0
Month              0
Year               0
InitialTime        0
Increment          0
TimeControl        0
Opening            0
ECO                0
Number_of_Moves    0
dtype: int64

In [ ]:
duplicate_rows_df = df[df.duplicated()]
print("Number of duplicate rows: ", duplicate_rows_df.shape)